# Costal Systems Notebook 2b: Astronomical Tidal Constituents

In Chapter 3 of the book and slides, we learned that the tide is generated through the interplay between gravitational attraction forces in the Earth-Moon and Earth-Sun systems respectively. In this notebook we will explore these concepts in a more interactive way, with some exercises, visualizations, and code. We will not repeat the whole theory, however, so make sure you have followed the lectures for this week and read the relevant pages in chapter 3 of the book.

Earth has continents, oceans with varying water depths, and a complex gravitational field, hence the tidal signals we observe around the planet are complex. Fortunately, we can break them down into multiple harmonic components, called ***tidal constituents***. These can either be diurnal, semi-diurnal, short-, or long-period constituents. Each constituent is characterized by an amplitude, frequency and phase, and individually looks like a smooth sin/cosine curve. When we sum up all the individual components we observe signal beating resulting in complex tidal signals with spring-neap cycles, daily inequalities, and longer-term variations.

A table of principal tidal constituents (Table 3.5) is provided below.

|Tidal constituents | Name | Equil. <br> Amplitude [m] | Period [h] |
|-|-|-|-|
| **Semi-diurnal** |
| Principal lunar | M2 | 0.24 | 12.42 |
| Principal solar | S2 | 0.11 | 12.00 |
| Lunar elliptical | N2 | 0.046 | 12.66 |
| Lunar-solar declinational | K2 | 0.031 | 11.97 |
| **Diurnal** |
| Lunar-solar declinational | K1 | 0.14 | 23.93 |
| Principal lunar | O1 | 0.10 | 25.82 | 
| Principal solar | P1 | 0.047 | 24.07 | 
| Lunar elliptical | Q1 | 0.019 | 26.87 |
| **Long period** |
| Fortnightly | Mf | 0.042 | 327.9 |
| Monthly | Mm | 0.022 | 661.3 | 
| Semi-annual | Ssa | 0.019 | 4383 | 

<br><br>

In [9]:
# Run this to import modules

import pathlib
from pathlib import Path
import sys
from warnings import filterwarnings
import ipywidgets as widgets
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
from datetime import datetime, timedelta
from IPython.display import display, Image
import math

cwd = pathlib.Path().resolve()
#proj_dir = cwd.parent  # this is the root of the CoastalCodeBook
DATA_DIR = proj_dir / "data"
#image_path = '/Users/mpupicvurilj/Library/CloudStorage/OneDrive-DelftUniversityofTechnology/Documents/TU Delft/Coastal Systems/Model points_globe.png'

NameError: name 'proj_dir' is not defined


<br><br>
## 1. Visualisation of Astronomical Constituents

Now that we are more familiar with the principal tidal constituents, we will visualise them at four different locations for better understanding. To accomplish this, we will use the [FES2014 Global Tide data](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes/description-fes2014.html), which provides amplitude and phase information for 34 tidal constituents, distributed on 1/16˚ grids. At the end of this codebook, you can find a short script on how to load the FES2014 dataset by yourselves.

First, let's have a look at the amplitudes of the constituents at each location. They are proved in the table below, in cm. Can you already determine which components will be dominant at the different locations? Also, take a look at the main lunar **M2** and solar **S2** components. In chapter 3 we determined that the moon is responsible for about 70% of the tidal mechanism, while the sun contributes only about 30%, meaning that the lunar tide should be around 2.3 times the solar tide. Does that apply at the four locations? If not, can you think of reasons why the ratio would be different?

|Tidal amplitudes [cm]| M2 | S2 | N2 | K2 | K1 | O1 | P1 |  Q1 |
|----------------|:--:|:--:|:--:|----|:--:|:--:|----|:---:|
| North Sea   | 26 | 4 | 3  |  2 | 9 | 12  |  4 |  4 |
| North Atlantic | 39 | 9 | 9  |  3 | 7 | 5  |  2  |  1 |
| South Atlantic | 13 | 7 | 4  |  2 | 5 | 8  |  1.4  |  2 |
| Indian Ocean | 44 |  26 | 8  | 7 | 3.5 | 3  | 1 |  1 |


<br> In the figure below, you can see where are the four model points located.

![image](book/assignments/figures/02_model_points_globe.png)


<br><br><br>
Let's now visualise the constituents! Take a look at the function below and run the cell, we will need it in the next steps.

In [4]:
### Functions
def harmonic(a, phi, w, t):
    """
    This function calculates the value of a harmonic function at a given time t.

    Parameters:
    a (float):      amplitude of the harmonic function.
    phi (float):    phase angle of the harmonic function.
    w (float):      angular frequency of the harmonic function.
    t (float):      time at which the function is evaluated.

    Returns:
    float:          the value of the harmonic function at the given time.
    """
    return a * np.cos(w * t + phi)

<br> <br><br>
In the code below we load the data (phase and amplitude of each constituent) and plot the tidal signal at each location. For each location, the individual components are plotted (upper plot, colors) and then the combined tidal signal (lower plot, black). It is an interactive plot, you can adjust the plotted time range using the slider (from 1 day to 1 year), and you can select which tidal constituents should be plotted with the tick boxes. This way you can play around with the different constituents and see what kind of signal you get.

In [5]:
# Tidal components and their angular frequencies (2pi divided by the period in days (= hours * 24)). Periods can be found in Table 3.5.
comps = {
    "M2": [2 * np.pi / 12.42 * 24, "C0"],  
    "S2": [2 * np.pi / 12 * 24, "C1"],   
    "N2": [2 * np.pi / 12.66 * 24, "C7"],
    "K2": [2 * np.pi / 11.97 * 24, "C2"],   
    "K1": [2 * np.pi / 23.93 * 24, "C3"], 
    "O1": [2 * np.pi / 25.82 * 24, "C4"],  
    "P1": [2 * np.pi / 24.07 * 24, "C5"],  
    "Q1": [2 * np.pi / 26.87 * 24, "C6"],  
}

locs = ["North Sea", "North Atlantic", "South Atlantic", "Indian Ocean"]

# List of the tidal components
list_comp = [comp for comp in comps]

# Time window for which we will compute the tidal components. We will use one year, 
# but in the graph we can manually adjust the plotted time window
t = np.linspace(0, 365, 20000)

# Create a time array using Julian days since 1970
start_date = datetime(1950, 1, 1)
end_date = start_date + timedelta(days=365)
num_points = 20000
julian_days = np.linspace(0, (end_date - start_date).days, num_points)

# Convert Julian days to datetime
time_array = start_date + np.array([timedelta(days=float(day)) for day in julian_days])


In [6]:
# Now we load the data into a dictionary so that we have one dataframe for each component (M2, S2, ...)
data_dir_path = Path(DATA_DIR)
tide = {}
for comp in comps.keys():
    #fp = (DATA_DIR / ("02_" + comp)).with_suffix(".p")
    fp = data_dir_path / ("02_" + comp + ".p")
    tide[comp] = pd.read_pickle(fp)

### Plotting widgets to make an interactive plot
# Time range slider
range_slider = widgets.IntRangeSlider(
    value=[0, int(max(t))],
    min=int(min(t)),
    max=int(max(t)),
    description="Time [d]",
    layout=widgets.Layout(width='50%')
)

# Tick boxes to select tidal components that should be plotted
M2 = widgets.Checkbox(description="M2")
S2 = widgets.Checkbox(description="S2")
N2 = widgets.Checkbox(description="N2")
K2 = widgets.Checkbox(description="K2")
K1 = widgets.Checkbox(description="K1")
O1 = widgets.Checkbox(description="O1")
P1 = widgets.Checkbox(description="P1")
Q1 = widgets.Checkbox(description="Q1")

comp_boxes = [widgets.Checkbox(description=comp) for comp in comps.keys()]

# Now we plot for each location a plot with the individual components and a plot for the total tidal water level
def plot(lims_x_axis, M2, S2, N2, K2, K1, O1, P1, Q1):
    height_ratios = [1, 1, 1.5, 1]
    fig, axs = plt.subplots(4, 2, sharex=True, figsize=(15, 8), constrained_layout=False)
    ax = axs.flatten()
    
    # We first create a dictionary with the tidal components that should be plotted,
    # based on the selected tick boxes (if a box is sleected the value for that compnent is True)
    comp2plot = dict()
    if M2 == True:
        comp2plot[list(comps.keys())[0]] = list(comps.values())[0]
    if S2 == True:
        comp2plot[list(comps.keys())[1]] = list(comps.values())[1] 
    if N2 == True:
        comp2plot[list(comps.keys())[2]] = list(comps.values())[2]
    if K2 == True:
        comp2plot[list(comps.keys())[3]] = list(comps.values())[3]
    if K1 == True:
        comp2plot[list(comps.keys())[4]] = list(comps.values())[4]
    if O1 == True:
        comp2plot[list(comps.keys())[5]] = list(comps.values())[5]
    if P1 == True:
        comp2plot[list(comps.keys())[6]] = list(comps.values())[6]
    if Q1 == True:
        comp2plot[list(comps.keys())[7]] = list(comps.values())[7]

    
    # Now we loop through the four locations (i) and the sub plots (j). Each location has two subplots, 
    # the upper showing the selected tidal components and the lower showing the combined tidal signal 
    # (sum of components).
    for i, j in zip(range(len(locs)), [0, 1, 4, 5]):
        
        # dictionary to store the harmonics
        harms = {}

        # Loop through the selected tidal components
        for comp in comp2plot.keys():

            # Store data for current component in temporary numpy array
            temp = tide[comp][tide[comp].index == locs[i].lower()].to_numpy()[0]

            # Use harmonic() to compute the harmonic funciton of the component
            harms[comp] = harmonic(temp[3], temp[2], comp2plot[comp][0], t)

            # Plot each harmonic component in the upper subplot
            ax[j].plot(time_array, harms[comp], "-", linewidth=1, c=comp2plot[comp][1])

        # Plot the sum of the selected constituents in the lower subplot
        if comp2plot:
            ax[j + 2].plot(time_array, sum([harms[comp] for comp in comp2plot.keys()]), "k", linewidth=1)

        # Axis settings
        x1 = datetime(1950,1,1) + timedelta(days=lims_x_axis[0])
        x2 = datetime(1950,1,1) + timedelta(days=lims_x_axis[1])
        ax[j].set_xlim([x1,x2])
        ax[j].set_title(locs[i], fontweight='bold')

    # Axis labels
    fig.text(0.5, -0.05, 'Time', ha='center', va='center', fontsize=14)
    fig.text(-0.02, 0.5, 'Tidal elevation [cm]', ha='center', va='center', rotation='vertical', fontsize=14)

    # Legend entries
    list_comp = [comp for comp in comp2plot]
    empty_str = [str("_nolegend_") for comp in comp2plot]
    list_comp += empty_str
    list_comp += ["Sum of components"]
    
    # Create legend
    fig.legend(
        list_comp,
        ncol=len(list_comp),
        bbox_to_anchor=(1, -0.1),
        fontsize=14,
    )

    # Tight plot layout format
    plt.tight_layout()

# Filter potential warnings for a clean output
filterwarnings("ignore", category=UserWarning)

# Create the figure
figure = widgets.interactive(plot,lims_x_axis=range_slider, M2=M2, S2=S2, N2=N2, K2=K2, K1=K1, O1=O1, P1=P1, Q1=Q1)
figure.layout.height = '100%'
time = figure.children[0]
controls1 = widgets.HBox(figure.children[1:5])
controls2 = widgets.HBox(figure.children[5:-1])
out = figure.children[-1]
widgets.VBox([time, controls1, controls2, out])


<br><br>
## 2. Tidal Beating

Using the knowledge gained from Chapter 3 of the textbook and the interactive figure above, try to answer the questions below. You can use the next cell as a calculator.



In [7]:
# Write your code here to get answers to questions 1 and 2. Use the table of principal constituents.

# T_group = 2*math.pi / (omega2-omega1) #equation 3.24b from the textbook
# omega2 = ?
# omega1 = ?
# T = ? [days]

print(T)

NameError: name 'T' is not defined

In [8]:
# Run this block to get questions

from tide_initialize import init

init()



1. In a semi-diurnal environment, spring tide occurs for tidal constituents A and B every N days. Set the time range to around 30 days, which phenomenon can you detect when looking at the combined signal of these two constituents?




2. In a diurnal environment, spring tide occurs for tidal constituents C and D every M days. What is the main difference to the signal from question 1?




3. Strongest semi-diurnal tides are in the months M1 and M2, as can be seen from adding constituents E and F.




4. Strongest diurnal tides are in the months M3 and M4, as can be seen from adding constituents G and H.




5. The combinations we looked at above have periods of up to 2 weeks. But we also know that there are much longer variations. Set the time-range to 1 year and plot the combinations S2/K2 and K1/P1. What kind of combined signal can you see now?


6. Finally activate all eight principal components and analyse the combined signals. What are the dominant components at each location?



In [13]:
# ## Script for loading FES2014 data

# #%% Load tides
# import xarray as xr
# import pickle
# import numpy as np
# from os.path import join
# import pandas as pd

# path = 'path_to_data/ocean_tide_extrapolated/' #Put your path

# #%% Load data 
# # You can choose between 34 tidal constituents
# comps = ['m2', 's2','n2', 'k2', 'k1', 'o1', 'p1', 'q1',
#          'm4', 'mf', 'mm', 'ssa', 'm6', 's4', 'mn4'
#         ]

# # Resolution of fes2014 is 0.0625x0.0625 (lat,lon), we choose a few 
# # locations and the closest model points. You can choose some
# # other locations
# locs = {
#     'scheveningen'  :   [52.125, 4.25], #lat, lon
#     'galveston'     :   [29.25, -94.6875],
#     'rio de janeiro':   [-23, -43.125],
#     'jakarta'       :   [-5.8125, 106.8125],
#     'north sea' :  [52.375, 3.25],
#     'north atlantic' : [36.25, -64],
#     'south atlantic' : [-30.8125, -37.25], 
#     'indian ocean' : [-5.8125, 77.25],

# }

# tide = {}

# # Extract constituents from FES2014 and store pickle files
# for comp in comps:
#     data = xr.open_dataset(join(path, comp + '.nc'))
#     data.coords['lon'] = (data.coords['lon'] + 180) % 360 - 180 #lon=[0,360]

#     # tide[comp] = pd.DataFrame(columns=)
#     temp = {}

#     for loc in locs.keys():
#         temp[loc] = (data.sel(lat=locs[loc][0],lon=locs[loc][1])
#         .to_dataframe()
#         .drop(['lat_bnds', 'lon_bnds', 'crs'], axis=1)
#         .drop(1)
#         .assign(place=loc)
#         .set_index('place', drop=True)
#     )

#     tide[comp] = pd.concat(temp.values())
#     tide[comp].to_pickle('path_to_data/02_%s.p' % comp) #Put the path to your folder where you want your constituents saved
    